# Grad-CAM application in ResTS and EfficientNetB0

In [ ]:
# Importações principais
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image

In [ ]:
# Função de pré-processamento para ResTS (usando Xception)
def preprocess_for_ResTS(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.xception.preprocess_input(img_array)

# Função de pré-processamento para EfficientNetB0
def preprocess_for_EfficientNet(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.efficientnet.preprocess_input(img_array)


In [ ]:
def load_and_preprocess_image(img_path, model_type="ResTS"):
    if model_type == "ResTS":
        return preprocess_for_ResTS(img_path)
    elif model_type == "EfficientNetB0":
        return preprocess_for_EfficientNet(img_path)

In [ ]:
# Carregar modelo ResTS
ResTS_model = tf.keras.models.load_model('../models/ResTS_trained_model')

# Carregar modelo EfficientNetB0 diretamente
EfficientNetB0_model = tf.keras.models.load_model('../models/EfficientNetB0_trained_model.h5')

In [ ]:
def compute_gradcam(model, img_array, layer_name, output_index=0):
    grad_model = Model(inputs=[model.inputs], outputs=[model.get_layer(layer_name).output, model.output[output_index]])
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = np.argmax(predictions[0])
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    cam = np.dot(conv_outputs, pooled_grads[..., tf.newaxis]).numpy()
    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (img_array.shape[2], img_array.shape[1]))
    cam = cam - np.min(cam)
    cam = cam / np.max(cam)
    return cam, class_idx

In [ ]:
def overlay_gradcam(img_path, cam):
    img = cv2.imread(img_path)
    cam = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    cam = cv2.addWeighted(img, 0.5, cam, 0.5, 0)
    return cam

def display_gradcam(model, img_path, layer_name, model_type="ResTS", output_index=0):
    img_array = load_and_preprocess_image(img_path, model_type=model_type)
    cam, class_idx = compute_gradcam(model, img_array, layer_name, output_index=output_index)
    overlay_img = overlay_gradcam(img_path, cam)
    plt.imshow(cv2.cvtColor(overlay_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [ ]:
# Caminho da imagem a ser testada
img_path = 'path_to_test_image.jpg'

# Grad-CAM para ResTeacher (primeira saída do ResTS)
print("ResTeacher Grad-CAM:")
display_gradcam(ResTS_model, img_path, layer_name="block14_sepconv2_act", model_type="ResTS", output_index=0)

# Grad-CAM para ResStudent (segunda saída do ResTS)
print("ResStudent Grad-CAM:")
display_gradcam(ResTS_model, img_path, layer_name="block14_sepconv2_act", model_type="ResTS", output_index=1)

# Grad-CAM para EfficientNetB0
print("EfficientNetB0 Grad-CAM:")
display_gradcam(EfficientNetB0_model, img_path, layer_name="top_conv", model_type="EfficientNetB0")